In [3]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [7]:
from src.ingestion.load_kaggle import load_data

df = load_data("../data/raw/creditcard.csv")
df.head()


,transaction_id,ts,user_id,amount,merchant,country,device_id,payment_method,label
0,0,2024-01-01 00:00:00,63,149.62,amazon,CO,1388,credit,0
1,1,2024-01-01 00:00:01,229,2.69,walmart,CO,4916,debit,0
2,2,2024-01-01 00:00:02,2134,378.66,amazon,BR,5678,credit,0
3,3,2024-01-01 00:00:03,1844,123.50,uber,BR,8607,credit,0
4,4,2024-01-01 00:00:04,2169,69.99,amazon,BR,6274,debit,0


In [8]:
df.dtypes

transaction_id            object
ts                datetime64[ns]
user_id                   object
amount                   float64
merchant                  object
country                   object
device_id                 object
payment_method            object
label                      int64
dtype: object